# Distributed XGBoost (CPU)

woohoo!

In [ ]:
!pip install --upgrade dask distributed adlfs xgboost fastparquet

## Get Workspace

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

## Get Data

The data...

start a distributed Client

In [ ]:
from distributed import Client

c = Client()
c

initialize the Pythonic filesystem

In [ ]:
from adlfs import AzureBlobFileSystem

container_name = "malware"
storage_options = {"account_name": "azuremlexamples"}

fs = AzureBlobFileSystem(**storage_options)
fs

list the processed (partitioned) files

In [ ]:
files = fs.ls(f"{container_name}/processed")
files

read data into a (dask) dataframe

In [ ]:
import pandas as pd
import dask.dataframe as dd

for f in files:
    if "train" in f:
        df_train = dd.read_parquet(f"az://{f}", storage_options=storage_options)
    elif "test" in f:
        df_test = dd.read_parquet(f"az://{f}", storage_options=storage_options)

df_train

## Exploratory Data Analysis (EDA)

Explore the data...

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
%%time
df_train.describe().compute()

In [ ]:
df_train["HasDetections"].compute().hist()

## Data Preparation

Prepare data for ML

throw out non-numeric columns...

In [ ]:
for col in df_train.dtypes:
    # print(col)
    pass

cols = [col for col in df_train.columns if df_train.dtypes[col] != "object"]
cols

In [ ]:
X = df_train[cols].drop("HasDetections", axis=1).values
X

In [ ]:
y = df_train["HasDetections"]
y

## Train XGBoost

In [ ]:
import xgboost as xgb

# dtrain = xgb.dask.DaskDMatrix(c, X, y)
# dtrain

In [ ]:
model = xgb.dask.train(c)

## Get Data

We'll use ...